In [2]:
from PIL import Image

import numpy as np
import tensorflow as tf

2024-05-10 21:41:10.132090: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-10 21:41:10.934932: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
img = Image.open('./monet_jpg/0a5075d42a.jpg')

In [4]:
print(img.format, img.size, img.mode)

JPEG (256, 256) RGB


In [5]:
np_data = np.asarray(img)

print(type(np_data), np_data.shape)

<class 'numpy.ndarray'> (256, 256, 3)


In [6]:
tfrec_dataset = tf.data.TFRecordDataset("./monet_tfrec/monet00-60.tfrec")

2024-05-10 21:41:12.143377: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-10 21:41:12.365117: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-10 21:41:12.365270: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [7]:
feature_description = {
    'image_raw': tf.io.FixedLenFeature([], tf.string),
}

def _parse_function(example_proto):
    return tf.io.parse_single_example(example_proto, feature_description)

In [8]:
dataset = tfrec_dataset.map(_parse_function)

In [9]:
def decode_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32)

    return image

In [10]:
dataset = dataset.map(lambda x: decode_image(x["image_raw"]))

In [11]:
dataset

<_MapDataset element_spec=TensorSpec(shape=(None, None, 3), dtype=tf.float32, name=None)>

In [12]:
np.prod((32, 32, 3))

3072